# Einführung: Retrieval-Augmented Generation (RAG)

Dieses Notebook demonstriert die Grundstruktur eines Retrieval-Augmented Generation (RAG) Systems.  
RAG kombiniert die Stärken von Retrieval- und Generierungsmodellen, um präzise Antworten auf Fragen zu liefern, die auf spezifischen Dokumenten basieren.
Retrieval-Modelle durchsuchen große Dokumentensammlungen, um relevante Informationen zu finden, während Generierungsmodelle diese Informationen nutzen, um kohärente Antworten zu formulieren.

Praktisch kann ein solches System genutzt werden, um Fragen zu beantworten, die auf einem bestimmten Dokument basieren, ohne dass das zugrunde liegende Sprachmodell neu trainiert werden muss.

## Aufbau eines RAG-Systems
Prinzipiell besteht ein RAG-System aus zwei Hauptkomponenten:
1. **Retrieval**: Hierbei werden relevante Abschnitte aus einem Dokument oder einer Sammlung von Dokumenten abgerufen, die für die Beantwortung der gestellten Frage nützlich sein könnten. Hierfür werden Dokumente in kleine Abschnitte ("Chunks") unterteilt und in einem Vektor-Datenbankindex gespeichert.  Der Index wird erstellt, indem die Abschnitte in Vektoren umgewandelt werden, die dann in der Datenbank gespeichert werden. Wenn eine Frage gestellt wird, wird der Index durchsucht, um die relevantesten Abschnitte zu finden.
2. **Generierung**: Basierend auf den abgerufenen Informationen generiert ein Sprachmodell eine Antwort auf die gestellte Frage.


## Ziel des Notebooks 
Fragen zu benutzerdefinierten Dokumenten beantworten – **ohne das Modell neu zu trainieren**.

## Voraussetzungen
- Python 3.8 oder höher
- Ein Ordner mit PDF-Dokumenten, die Sie verwenden möchten (z.B. `./data`)


In [2]:
# --- Benötigten Pakete installieren ---

!pip install -q pymupdf        # Für PDF-Text-Extraktion
!pip install -q numpy          # Für Cosine Similarity & Vektorberechnungen
!pip install -q litellm        # Für Zugriff auf Embedding- & Sprachmodelle via API
!pip install -q langchain      # Für die Verwaltung von Embeddings und Modellen

In [ ]:
# --- Imports ---

import os                 
import fitz # PyMuPDF          
import numpy as np        
import litellm
from langchain.text_splitter import RecursiveCharacterTextSplitter  
from typing import List, Tuple

In [ ]:
# --- OpenAI API Key setzen (über Umgebungsvariable) ---
os.environ["OPENAI_API_KEY"] = "sk-..."  # TODO: Ersetze durch deinen eigenen API-Schlüssel. Dieser API-Key kann auf https://platform.openai.com/account/api-keys generiert werden. 

# Alternativ kann auch Groq oder ein anderer Provider verwendet werden. Eine komplette Übersicht gibt es auf https://docs.litellm.ai/docs/providers.  
# Cohere bietet kostenlose API Keys mit einer Token-Begrenzung an https://docs.cohere.com/v2/docs/rate-limits 

## 📄 Schritt 1: PDF-Dokumente einlesen

Zuerst werden PDF-Dateien mit dem Python-Paket `fitz` (PyMuPDF) in reinen Text umgewandelt. Dafür definieren wir eine Funktion `extract_text_from_pdfs`, die alle PDF-Dateien in einem angegebenen Verzeichnis liest und den Text extrahiert.

In [ ]:
def extract_text_from_pdf_folder(pdf_folder_path):
    all_text = ""
    
    # Über alle PDF-Dateien im Ordner iterieren
    for filename in os.listdir(pdf_folder_path):
        if filename.lower().endswith(".pdf"):
            file_path = os.path.join(pdf_folder_path, filename)
            # TODO: Öffne die PDF-Datei mit fitz.open(file_path) als doc
            for page in doc:
                # TODO: Verwende .get_text(), um Text zu extrahieren und zu all_text hinzuzufügen
                pass
    
    return all_text

Loaded text from folder './pdfs', showing preview:

Polymer Solubility Prediction Using Large
Language Models
Published as part of ACS Materials Letters special issue “Machine Learning for Materials Chemistry”.
Sakshi Agarwal, Akhlak Mahmood, and Rampi Ramprasad*
Cite This: ACS Materials Lett. 2025, 7, 2017−2023
Read Online
ACCESS
Metrics & More
Article Recommendations
*
sı
Supporting Information
ABSTRACT: Traditional approaches in polymer informatics often require labor-intensive data curation, time-
consuming preprocessing such as fingerprinting, and choosing suitable learning algorithms. Large language models
(LLMs) represent a compelling alternative by addressing these limitations with their inherent flexibility, ease of use,
and scalability. In this study, we propose a novel approach utilizing fine-tuned LLMs to classify solvents and
nonsolvents for polymers, a property critical to polymer synthesis, purification, and diverse applications. Our results
show that fine-tuned GPT-3.5 

Nun können wir die Funktion zum Extrahieren von Text aus PDF-Dateien testen:

In [1]:
# --- Aufruf der Extraktion ---
pdf_folder = "..."  # TODO: Gib den Ordnerpfad mit den PDFs an
raw_text = extract_text_from_pdf_folder(pdf_folder)

NameError: name 'extract_text_from_pdf_folder' is not defined

Und uns den extrahierten Text anzeigen lassen

In [ ]:
print(raw_text[1000]) 

## ✂️ Schritt 2: Text in Chunks zerlegen

Der extrahierte Text wird nun in kleinere, überlappende Abschnitte (*Chunks*) aufgeteilt.  
Hierfur verwenden wir eine Methode die `RecursiveCharacterTextSplitter` genannt wird. Diese Methode teilt den Text in kleinere Abschnitte auf, die für die spätere Verarbeitung durch das Retrieval-Modell geeignet sind. Die Chunks werden so erstellt, dass sie eine maximale Länge haben und überlappende Teile enthalten, um sicherzustellen, dass wichtige Informationen nicht verloren gehen. Die Abschnitte werden hierbei erstellt indem der Text an einer definierten Liste von Trennzeichen (wie Absätzen oder Sätzen) aufgeteilt wird. Diese Liste wird durchgegangen bis der Text in kleinere Abschnitte zerlegt ist, die eine maximale Länge nicht überschreiten.
Mehr Informationen dazu können in der [LangChain Dokumentation](https://python.langchain.com/docs/how_to/recursive_text_splitter/) gefunden werden.

Diese Einheiten können später effizient eingebettet und durchsucht werden.


In [ ]:
# --- Text in überlappende Chunks aufteilen ---

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=___,       # TODO: Wähle sinnvolle Chunk-Größe
    chunk_overlap=___     # TODO: Wähle Überlappung
)

chunks = text_splitter.split_text()  # TODO: Gib den zu chunkenden Text ein
print(f"{len(chunks)} Chunks erstellt.")

260 chunks created.


## 🔢 Schritt 3: Chunks embedden

Nun werden die erzeugten Chunks in numerische Vektoren (*Embeddings*) umgewandelt.  
Dies erfolgt mithilfe der OpenAI API über `LiteLLM`.


In [ ]:
# Beispielkonfiguration (OpenAI, austauschbar)
litellm_model = "openai/embedding-3-small"

# Liste von Embeddings vorbereiten
embeddings = []

for chunk in chunks:
    response = litellm.completion( 
        model=litellm_model,
        input=chunk,
        api_type="embedding"
    )
    embeddings.append(response["data"][0]["embedding"]) 

print(f"{len(embeddings)} Embeddings erstellt.")

## 🧠 Schritt 4: Erstellung eines Vektorstores

Die Embeddings und ihre zugehörigen Textabschnitte werden in einem einfachen Vektorstore gespeichert.  
Dazu erstellen wir eine **Liste von Paaren** bestehend aus:

- einem Embedding  
- dem zugehörigen Textabschnitt

➡️ Dies erlaubt später eine **schnelle semantische Suche**.

In [ ]:
# --- Einfache Speicherstruktur für Embeddings + zugehörige Chunks ---

vectorstore: List[Tuple[List[float], str]] = list(zip(___, ___)) # TODO: Embeddings + zugehörige Text Chunk definieren

## 🔍 Schritt 5: Retrieval der relevanten Textabschnitte

Um zu einer Nutzeranfrage passende Textstellen zu finden, berechnen wir die **Cosine Similarity**  
zwischen dem Embedding der Frage und allen gespeicherten Embeddings.

In [ ]:
# --- Cosine Similarity berechnen ---
def cosine_similarity(embedding_1: np.ndarray, embedding_2: np.ndarray) -> float:
    return np.dot(embedding_1, embedding_2) / (np.linalg.norm(embedding_1) * np.linalg.norm(embedding_2))

Nun sollen die **k ähnlichsten Chunks** zur Nutzeranfrage (`query`) gefunden und zurückgegeben werden.

In [ ]:
# --- Ähnlichste Chunks zur Nutzerfrage finden ---
def retrieve_top_k(query: str, k: int = 3) -> List[str]:
    response = litellm.completion(
        model=litellm_model,
        input=___,      # TODO: Ersetze durch die Nutzeranfrage
        api_type=___    # TODO: API-Typ definieren
    )
    query_embedding = np.array(response["data"][0]["embedding"])

    # Ähnlichkeit mit allen gespeicherten Embeddings berechnen
    scored_chunks = []
    for embedding, text in vectorstore:
        embedding = np.array(embedding)
        score = cosine_similarity(___, ___) # TODO: Cosine Similarity korrekt aufrufen
        scored_chunks.append((score, text))

    # Chunks nach Score absteigend sortieren
    scored_chunks = sorted(
        ___,                      # TODO: Liste der Scoring-Ergebnisse einsetzen
        key=lambda x: x[0],       # Sortiere nach dem ersten Element im Tupel = Score
        reverse=True              # Höchste Scores zuerst
    )

    # Texte der Top-k Ergebnisse zurückgeben
    top_chunks = []

    for i in range(min(___, len(___))):  # TODO: Ersetze beide ___ mit der gewünschten Anzahl an Ergebnissen und der Länge der Liste scored_chunks
        top_chunks.append(scored_chunks[i][1])  
        
    return top_chunks

## 💬 Schritt 6: Nutzeranfrage stellen und Antwort generieren

Die Nutzerfrage wird zunächst ebenfalls in ein Embedding umgewandelt.  
Danach werden die semantisch ähnlichsten Chunks aus dem Vektorstore geladen.  
Diese bilden den **Kontext**, den das Sprachmodell (z.B. GPT-4) verwendet, um eine Antwort zu generieren.

In [ ]:
# --- Nutzerfrage stellen ---
query = "..."  # TODO: Gib hier deine Frage ein

# --- Passende Chunks aus selbstgebauter Vektorstore abfragen ---
top_chunks = retrieve_top_k(___, ___)  # TODO: Argumente der Abfragefunktion definieren

# --- Prompt vorbereiten ---
retrieved_context = "\n\n".join(top_chunks)

# --- Prompt + Frage an Sprachmodell übergeben (z.B. GPT-4 via LiteLLM) ---
response = litellm.completion(
    model="gpt-4",  # TODO: Modell ggf. anpassen
    messages=[
        {"role": "system", "content": "Beantworte Fragen basierend auf den folgenden Textauszügen."},
        {"role": "user", "content": f"Textauszüge: {___}\n\nFrage: {___}"} # TODO: Ähnliche Textauszüge und Nutzteranfrage definieren
    ]
)

# --- Antwort anzeigen ---
print("Antwort:")
print(response["choices"][0]["message"]["content"])

In diesem Tutorial hast du Schritt für Schritt ein einfaches Retrieval-Augmented Generation (RAG) System aufgebaut.

Du hast gelernt:

- wie man Dokumente in Text umwandelt,
- wie man diesen Text in verarbeitbare Chunks aufteilt,
- wie man eigene Embeddings erzeugt,
- und wie man eine semantische Suche selbst implementiert.

Anschließend konntest du mit Hilfe eines Sprachmodells Fragen zu beliebigen Dokumenten beantworten.

🔧 Dieses Grundgerüst lässt sich nun beliebig erweitern – z.B. mit:
- Vektor-Datenbanken wie FAISS, Chroma oder Weaviate
- lokalen Sprachmodellen (z.B. über Ollama, Hugging Face)
- anderen Datenquellen (z.B. HTML, CSV, Notizen, Mails)